# Collaborative Filtering Recommender System
In this notebook, we'll implement collaborative filtering to build a recommender system for movies (and use it on two sample datasets from the course graded lab).  
The code here are based on my own implementations in the graded lab, organized and rewritten to be more succinct and clear.

### Tensors and operations in tensorflow
See this [link](https://www.tensorflow.org/tutorials/customization/basics) for tensors and their operations, as well as the conversion between tensors and numpy arrays.

## Tools

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

## Collaborative Filtering Algorithm

We'll be predicting the movie ratings as follows: for user $j$, predict rating for movie $i$ as:

$$
Y(i,j) = R(i,j) * (\vec{w}^{(j)} \cdot \vec{x}^{(i)} + b^{(j)})
$$

where $R(i,j) = 1$ if user $j$ has rated movie $i$, and $R(i,j) = 0$ if not.

We'll use gradient descent with the following cost function to learn the parameters $\mathbf{X}$, $\mathbf{W}$, and $\mathbf{b}$ collaboratively.

$$J({\mathbf{x}^{(0)},...,\mathbf{x}^{(n_m-1)},\mathbf{w}^{(0)},b^{(0)},...,\mathbf{w}^{(n_u-1)},b^{(n_u-1)}})= \left[ \frac{1}{2}\sum_{(i,j):r(i,j)=1}(\mathbf{w}^{(j)} \cdot \mathbf{x}^{(i)} + b^{(j)} - y^{(i,j)})^2 \right]
+ \underbrace{\left[
\frac{\lambda}{2}
\sum_{j=0}^{n_u-1}\sum_{k=0}^{n-1}(\mathbf{w}^{(j)}_k)^2
+ \frac{\lambda}{2}\sum_{i=0}^{n_m-1}\sum_{k=0}^{n-1}(\mathbf{x}_k^{(i)})^2
\right]}_{regularization}
$$

## Implementation

### Cost function
We'll implement the cost function with the following function:
- `cofi_cost_func`: compute cost for collaborative filtering

In [7]:
def cofi_cost_func(X, W, b, Y, R, lambda_):
    """
    Returns the cost for the collaborative filtering
    Vectorized for speed. Uses tensorflow operations to be compatible with custom training loop.
    Args:
      X (ndarray (num_movies,num_features)): matrix of item features
      W (ndarray (num_users,num_features)) : matrix of user parameters
      b (ndarray (1, num_users)            : vector of user parameters
      Y (ndarray (num_movies,num_users)    : matrix of user ratings of movies
      R (ndarray (num_movies,num_users)    : matrix, where R(i, j) = 1 if the i-th movies was rated by the j-th user
      lambda_ (float): regularization parameter
    Returns:
      J (float) : Cost
    """
    J = tf.math.reduce_sum((R * (tf.linalg.matmul(X, tf.transpose(W)) + b - Y))**2)
    reg = lambda_ * (tf.math.reduce_sum(W**2) + tf.math.reduce_sum(X**2))
    J = (J + reg) / 2
    
    return J

### Model training
We'll train the model with a custom training loop in tensorflow with the Adam optimizer:

In [8]:
# Parameters
n_m, n_u = Y.shape  # n_m: number of movies, n_u: number of users
n_f = 100  # number of features for each movie
alpha = 1e-1  # learning rate

# Set Initial Parameters (W, X), use tf.Variable to track these variables
# Randomly initalized parameters W, b, X
tf.random.set_seed(1234) # for consistent results
W = tf.Variable(tf.random.normal([n_u, n_f], dtype=tf.float64), name='W')
X = tf.Variable(tf.random.normal([n_m, n_f], dtype=tf.float64), name='X')
b = tf.Variable(tf.random.normal([1, n_u], dtype=tf.float64), name='b')

# Instatntiate on optimizer
optimizer = keras.optimizers.Adam(learning_rate = alpha)

NameError: name 'Y' is not defined

In [ ]:
iterations = 200
lambda_ = 1.

for iter in range(iterations):
    # Use TensorFlow’s GradientTape
    # to record the operations used to compute the cost 
    with tf.GradientTape() as tape:

        # Compute the cost (forward pass included in cost)
        cost_value = cofi_cost_func(X, W, b, Ynorm, R, lambda_)

    # Use the gradient tape to automatically retrieve
    # the gradients of the trainable variables with respect to the loss
    grads = tape.gradient( cost_value, [X,W,b] )

    # Run one step of gradient descent by updating
    # the value of the variables to minimize the loss.
    optimizer.apply_gradients( zip(grads, [X,W,b]) )

    # Log periodically.
    if iter % 20 == 0:
        print(f"Training loss at iteration {iter}: {cost_value:0.1f}")